In [ ]:
### 包依赖
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [ ]:
#全局变量
file_feature = r"clean_data"


In [ ]:
#模型构建
# feature_size                                 输入特征个数
# temporal_size                                时间步长度
# temporal_fc_hidden_size                      时间维度全连接层神经元个数
# feature_fc_hidden_size                       特征维度全连接层神经元个数
# lstm_layer                                   LSTM层个数
# cnn_kernel_size                              1d卷积层卷积核大小

class NewModel(nn.Module):
    def __init__(self, feature_size, temporal_size, temporal_fc_hidden_size, feature_fc_hidden_size, lstm_layer, cnn_kernel_size):
        super(NewModel, self).__init__()
        #参数初始化
        self.feature_size = feature_size
        self.temporal_size = temporal_size
        self.temporal_fc_hidden_size = temporal_fc_hidden_size
        self.feature_fc_hidden_size = feature_fc_hidden_size

        #模型初始化
        # Flatten, 用于展平数据
        self.flatten = nn.Flatten()
        
        #时间维度全连接层
        self.temporal_fc = nn.Sequential(
            nn.Linear(1, self.temporal_fc_hidden_size)
            nn.ReLU()
            nn.Linear(self.temporal_fc_hidden_size, self.temporal_fc_hidden_size)
        )
        
        #特征维度全连接层
        self.feature_fc = nn.Sequential(
            nn.Linear(1, self.feature_fc_hidden_size)
            nn.ReLU()
            nn.Linear(self.feature_fc_hidden_size, self.feature_fc_hidden_size)
        )

        #LSTM层
        self.feature_demention = self.temporal_fc_hidden_size
        self.cell_size = self.temporal_fc_hidden_size
        self.lstm_layer = lstm_layer
        self.lstm = nn.LSTM(input_size = self.feature_demention, hidden_size = self.cell_size, num_layers = self.lstm_layer, batch_first = True)
        
        #CNN层
        self.cnnin = self.feature_size
        self.cnn_kernel_size = cnn_kernel_size
        self.cnn_padding = (cnn_kernel_size - 1)/2
        self.conv1d = nn.Conv1d(in_channels = self.cnnin, out_channels = self.cnnin, kernel_size = self.cnn_kernel_size, padding = self.cnn_padding)
        
        #Sigmoid，用作非线性变换
        self.sigmoid = nn.Sigmoid()
        
        #归一化
        code
        
        #TA，时间注意力机制
        code
        
        #MHA，多头注意力机制
        code
        
        #全连接层
        code

        #Softmax，输出多类型概率
        code
        
    def forward(self, x):
        #时间维度的全连接层Temporal_FC
        #按列展平
        x = x.transpose(1,2)
        x2 = self.flatten(x)
        x2 = x2.flatten() #将batch_size维度也一并展平，形成1维tensor
        
        time_matrix = [] 
        time_tensor = []
        time_batch = []
        count = 1
        for i in x2:
            data = i
            data = data.unsqueeze(0)
            fc_out = self.temporal_fc(data)
            time_matrix.append(fc_out)
            if count % self.temporal_size == 0:
                matrix = torch.stack(time_matrix)
                time_tensor.append(matrix)
                time_matrix = []
            if count % (self.temporal_size * self.feature_size) == 0:
                tensor = torch.stack(time_tensor)
                time_batch.append(tensor)
                time_tensor = []
            count += 1
        time_batch = torch.stack(time_batch)
        # time_batich = [batch_size, feature_size, temporal_size, temporal_fc_hidden_size]

        #LSTM层
        #循环遍历每一个batch
        #特征数量对应torch中的batch_size
        #temporal_size对应torch中的sequence length
        #temporal_fc_hidden_size对应torch中的特征个数
        lstm_outs = []
        for f in range(time_batch.size(0):
            sub_tensor = time_batch[f]
            lstm_out, _ = self.lstm(sub_tensor) #此处输出的lstm_out的tensor与sub_tensor大小一致，可以进行矩阵加减
            lstm_outs.append(lstm_out)
        lstm_outs = torch.stack(lstm_outs)
        
        #时间维度的Add&Norm
        sigmoid_time = self.sigmoid(time_batch)
        sigmoid_lstm = self.sigmoid(lstm_outs)
        TA_in = sigmoid_time + sigmoid_lstm
        # TA_in: [batch_size, feature_size, temporal_size, temporal_fc_hidden_size]
        
        #时空注意力机制TA
        code
        
        #特征维度的全连接层Feature_FC
        #按行展平
        x1 = self.flatten(x)
        x1 = x1.flatten() #将batch_size维度也一并展平，形成1维tensor
        
        feature_matrix = [] 
        feature_tensor = []
        feature_batch = []
        count = 1
        for i in x1:
            data = i
            data = data.unsqueeze(0)
            fc_out = self.feature_fc(data)
            feature_matrix.append(fc_out)
            if count % self.feature_size == 0:
                matrix = torch.stack(feature_matrix)
                feature_tensor.append(matrix)
                feature_matrix = []
            if count % (self.feature_size * self.temporal_size) == 0:
                tensor = torch.stack(feature_tensor)
                feature_batch.append(tensor)
                feature_tensor = []
            count += 1
        feature_batch = torch.stack(feature_batch)
        # feature_batch = [batch_size, temporal_size, feature_size, feature_fc_hidden_size]
        
        #CNN层
        cnn1d_outs = []
        for f in range(feature_batch.size(0):
            #[batch_size, feature_size, sequence] conv1d在最后一个维度卷积
            sub_tensor = feature_batch[f]
            cnn1d_out = self.conv1d(sub_tensor) #此处输出的conv1d_out的形状与sub_tensor形状一致，可以进行矩阵加减
            cnn1d_outs.append(cnn1d_out)
        cnn1d_outs = torch.stack(cnn1d_outs)
        
        #特征维度的Add&Norm
        sigmoid_feature = self.sigmoid(feature_batch)
        sigmoid_cnn = self.sigmoid(cnn1d_outs)
        MHA_in = sigmoid_feature + sigmoid_cnn
        # MHA_in: [batch_size, temporal_size, feature_size, feature_fc_hidden_size]
        
        #多头注意力机制MHA
        code
        
        #Add
        code
        
        #全连接层FC + Softmax 得到输出
        code
        

![jupyter](./模型流程图.png)

In [2]:
class NewModel(nn.Module):
    def __init__(self, feature_size, temporal_size, temporal_fc_hidden_size, n):
        super(NewModel, self).__init__()

        self.feature_size = feature_size
        self.temporal_size = temporal_size
        self.temporal_fc_hidden_size = temporal_fc_hidden_size
        self.n = n

        
        self.flatten = nn.Flatten()
        
        self.temporal_fc = nn.Sequential(
            nn.Linear(1, self.temporal_fc_hidden_size),
            nn.ReLU(),
            nn.Linear(self.temporal_fc_hidden_size, self.n)
        )

        
    def forward(self, x):   
        #按列展平
        x = x.transpose(1,2)
        x2 = self.flatten(x)
        x2 = x2.flatten() #将batch_size维度也一并展平，形成1维tensor

        #时间维度的全连接层Temporal_FC
        time_matrix = [] 
        time_tensor = []
        time_batch = []
        count = 1
        for i in x2:
            data = i
            data = data.unsqueeze(0)
            fc_out = self.temporal_fc(data)
            time_matrix.append(fc_out)
            if count % 20 == 0:
                matrix = torch.stack(time_matrix)
                time_tensor.append(matrix)
                time_matrix = []
            if count % (self.temporal_size * self.feature_size) == 0:
                tensor = torch.stack(time_tensor)
                time_batch.append(tensor)
                time_tensor = []
            count += 1
        time_batch = torch.stack(time_batch)
        
        return time_batch

In [3]:
#创造数据
data = torch.randn(2,100,20)
model = NewModel(20,100,10,10)

model.train()
total_out = model(data)

In [4]:
total_out.shape

torch.Size([2, 100, 20, 10])

In [5]:
data

tensor([[[ 1.3896e+00,  2.5180e-01, -4.4530e-01,  ..., -2.6253e-01,
           3.1068e-01, -5.9803e-01],
         [ 1.9354e-01,  5.1342e-01,  4.9294e-02,  ...,  1.2007e+00,
          -2.1513e+00,  7.1522e-01],
         [ 4.7915e-01,  2.1839e-01,  3.8866e-01,  ...,  8.0404e-01,
           8.7054e-01,  2.6725e-01],
         ...,
         [ 8.7002e-02,  4.2814e-01,  5.6115e-01,  ..., -1.5945e-01,
          -1.9266e+00,  1.1306e-01],
         [-1.5509e+00,  7.9575e-02, -1.1222e+00,  ..., -5.1988e-01,
           3.2378e-01,  1.0980e-01],
         [ 1.1727e+00,  4.6077e-01, -6.1920e-01,  ..., -3.3946e-01,
          -6.0177e-01, -1.2405e+00]],

        [[ 1.6218e+00,  1.3591e+00, -1.6697e-01,  ...,  1.2692e+00,
          -1.0682e+00,  2.4145e+00],
         [ 7.3825e-01, -8.5672e-01, -3.2349e-01,  ..., -1.6664e-01,
           1.1788e-01, -7.2712e-02],
         [-1.3064e+00, -3.6120e-01, -4.7639e-01,  ...,  4.2178e-01,
           2.0579e+00, -9.3695e-01],
         ...,
         [ 2.8061e-01,  1

In [6]:
print(total_out.shape)
print(data.shape)

torch.Size([40, 100, 10])
torch.Size([2, 100, 20])
